In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import itertools
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import acf

import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv("/content/drive/MyDrive/Time Series Analysis/Data/train.csv")
df["date"] = pd.to_datetime(df['date'], format='%Y-%m-%d') # formatting the date

df_test = pd.read_csv("/content/drive/MyDrive/Time Series Analysis/Data/test.csv")
df_test["date"] = pd.to_datetime(df_test['date'], format='%Y-%m-%d') # formatting the date

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_final_arma_v2 = pd.DataFrame(columns=['id','date','store_nbr','family','onpromotion','pred'])

In [ ]:
df_stores = pd.read_csv('/content/drive/MyDrive/Time Series Analysis/Data/stores.csv')
df_stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [ ]:
#Fetching list of unique products and store numbers
family_list = df['family'].unique().tolist()
store_list = df['store_nbr'].unique().tolist()

comb_store_family = list(itertools.product(store_list, family_list))
len(comb_store_family)

1782

In [ ]:
df_merge = pd.merge(df, df_stores[['store_nbr','cluster']], how='left', on='store_nbr')
df_merge.head()

,id,date,store_nbr,family,sales,onpromotion,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,13
1,1,2013-01-01,1,BABY CARE,0.0,0,13
2,2,2013-01-01,1,BEAUTY,0.0,0,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,13
4,4,2013-01-01,1,BOOKS,0.0,0,13


In [ ]:
family_list

['AUTOMOTIVE',
 'BABY CARE',
 'BEAUTY',
 'BEVERAGES',
 'BOOKS',
 'BREAD/BAKERY',
 'CELEBRATION',
 'CLEANING',
 'DAIRY',
 'DELI',
 'EGGS',
 'FROZEN FOODS',
 'GROCERY I',
 'GROCERY II',
 'HARDWARE',
 'HOME AND KITCHEN I',
 'HOME AND KITCHEN II',
 'HOME APPLIANCES',
 'HOME CARE',
 'LADIESWEAR',
 'LAWN AND GARDEN',
 'LINGERIE',
 'LIQUOR,WINE,BEER',
 'MAGAZINES',
 'MEATS',
 'PERSONAL CARE',
 'PET SUPPLIES',
 'PLAYERS AND ELECTRONICS',
 'POULTRY',
 'PREPARED FOODS',
 'PRODUCE',
 'SCHOOL AND OFFICE SUPPLIES',
 'SEAFOOD']

In [ ]:
cluster_mapping = df_merge[['store_nbr',"cluster","family"]].groupby(['store_nbr',"cluster","family"]).count().reset_index()

In [ ]:
cluster_mapping.head()

,store_nbr,cluster,family
0,1,13,AUTOMOTIVE
1,1,13,BABY CARE
2,1,13,BEAUTY
3,1,13,BEVERAGES
4,1,13,BOOKS


In [ ]:
cluster_mapping[cluster_mapping["cluster"]==1]

,store_nbr,cluster,family
759,24,1,AUTOMOTIVE
760,24,1,BABY CARE
761,24,1,BEAUTY
762,24,1,BEVERAGES
763,24,1,BOOKS
...,...,...,...
886,27,1,POULTRY
887,27,1,PREPARED FOODS
888,27,1,PRODUCE
889,27,1,SCHOOL AND OFFICE SUPPLIES


In [ ]:
#providing custom inputs
list_df = cluster_mapping.values.tolist()
list_cluster_ar = []
for i in list_df:
  print(i)
  df_temp = df_merge[(df_merge['store_nbr']==i[0])&(df_merge['family']==i[2])]
  if df_temp["sales"].sum() > 0:
    pacf_test = pacf(df_temp['sales'].values,method='ywadjusted',nlags=20)
    df_pacf = pd.DataFrame(pacf_test, columns=['pacf_value'])
    df_pacf["abs_pacf"]= df_pacf["pacf_value"].abs()
    df_pacf.drop(df_pacf[df_pacf.abs_pacf < 0.05].index, inplace=True)
    df_pacf["difference"] = df_pacf.abs_pacf - 0.05
    df_pacf['cum_sum'] = df_pacf['difference'].cumsum()
    df_pacf['div'] = df_pacf['cum_sum'].diff()/df_pacf['cum_sum']
    df_pacf = df_pacf.loc[1:,:]
    list_index = df_pacf[(df_pacf["div"] >= df_pacf["div"].mean()) & (df_pacf.index > 7)].index.values.tolist()
    if len(list_index) > 0:
      i.append(list_index[0])
    else:
      i.append(7)
  else:
    i.append(1)
  list_cluster_ar.append(i)

[1, 13, 'AUTOMOTIVE']
[1, 13, 'BABY CARE']
[1, 13, 'BEAUTY']
[1, 13, 'BEVERAGES']
[1, 13, 'BOOKS']
[1, 13, 'BREAD/BAKERY']
[1, 13, 'CELEBRATION']
[1, 13, 'CLEANING']
[1, 13, 'DAIRY']
[1, 13, 'DELI']
[1, 13, 'EGGS']
[1, 13, 'FROZEN FOODS']
[1, 13, 'GROCERY I']
[1, 13, 'GROCERY II']
[1, 13, 'HARDWARE']
[1, 13, 'HOME AND KITCHEN I']
[1, 13, 'HOME AND KITCHEN II']
[1, 13, 'HOME APPLIANCES']
[1, 13, 'HOME CARE']
[1, 13, 'LADIESWEAR']
[1, 13, 'LAWN AND GARDEN']
[1, 13, 'LINGERIE']
[1, 13, 'LIQUOR,WINE,BEER']
[1, 13, 'MAGAZINES']
[1, 13, 'MEATS']
[1, 13, 'PERSONAL CARE']
[1, 13, 'PET SUPPLIES']
[1, 13, 'PLAYERS AND ELECTRONICS']
[1, 13, 'POULTRY']
[1, 13, 'PREPARED FOODS']
[1, 13, 'PRODUCE']
[1, 13, 'SCHOOL AND OFFICE SUPPLIES']
[1, 13, 'SEAFOOD']
[2, 13, 'AUTOMOTIVE']
[2, 13, 'BABY CARE']
[2, 13, 'BEAUTY']
[2, 13, 'BEVERAGES']
[2, 13, 'BOOKS']
[2, 13, 'BREAD/BAKERY']
[2, 13, 'CELEBRATION']
[2, 13, 'CLEANING']
[2, 13, 'DAIRY']
[2, 13, 'DELI']
[2, 13, 'EGGS']
[2, 13, 'FROZEN FOODS']
[2, 13, 'G

In [ ]:
list_df = cluster_mapping.values.tolist()
list_cluster_ma = []
for i in list_df:
  print(i)
  df_temp = df_merge[(df_merge['store_nbr']==i[0])&(df_merge['family']==i[2])]
  if df_temp["sales"].sum() > 0:
    acf_test = acf(df_temp['sales'].values,adjusted='True',nlags=20)
    df_acf = pd.DataFrame(acf_test, columns=['acf_value'])
    df_acf["abs_acf"]= df_acf["acf_value"].abs()
    df_acf.drop(df_acf[df_acf.abs_acf < 0.05].index, inplace=True)
    df_acf["difference"] = df_acf.abs_acf - 0.05
    df_acf['cum_sum'] = df_acf['difference'].cumsum()
    df_acf['div'] = df_acf['cum_sum'].diff()/df_acf['cum_sum']
    df_acf = df_acf.loc[1:,:]
    list_index = df_acf[(df_acf["div"] >= df_acf["div"].mean()) & (df_acf.index > 7)].index.values.tolist()
    if len(list_index) > 0:
      i.append(list_index[0])
    else:
      i.append(7)
  else:
    i.append(1)
  list_cluster_ma.append(i)

[1, 13, 'AUTOMOTIVE']
[1, 13, 'BABY CARE']
[1, 13, 'BEAUTY']
[1, 13, 'BEVERAGES']
[1, 13, 'BOOKS']
[1, 13, 'BREAD/BAKERY']
[1, 13, 'CELEBRATION']
[1, 13, 'CLEANING']
[1, 13, 'DAIRY']
[1, 13, 'DELI']
[1, 13, 'EGGS']
[1, 13, 'FROZEN FOODS']
[1, 13, 'GROCERY I']
[1, 13, 'GROCERY II']
[1, 13, 'HARDWARE']
[1, 13, 'HOME AND KITCHEN I']
[1, 13, 'HOME AND KITCHEN II']
[1, 13, 'HOME APPLIANCES']
[1, 13, 'HOME CARE']
[1, 13, 'LADIESWEAR']
[1, 13, 'LAWN AND GARDEN']
[1, 13, 'LINGERIE']
[1, 13, 'LIQUOR,WINE,BEER']
[1, 13, 'MAGAZINES']
[1, 13, 'MEATS']
[1, 13, 'PERSONAL CARE']
[1, 13, 'PET SUPPLIES']
[1, 13, 'PLAYERS AND ELECTRONICS']
[1, 13, 'POULTRY']
[1, 13, 'PREPARED FOODS']
[1, 13, 'PRODUCE']
[1, 13, 'SCHOOL AND OFFICE SUPPLIES']
[1, 13, 'SEAFOOD']
[2, 13, 'AUTOMOTIVE']
[2, 13, 'BABY CARE']
[2, 13, 'BEAUTY']
[2, 13, 'BEVERAGES']
[2, 13, 'BOOKS']
[2, 13, 'BREAD/BAKERY']
[2, 13, 'CELEBRATION']
[2, 13, 'CLEANING']
[2, 13, 'DAIRY']
[2, 13, 'DELI']
[2, 13, 'EGGS']
[2, 13, 'FROZEN FOODS']
[2, 13, 'G

In [ ]:
for i,j in zip(list_cluster_ar,list_cluster_ma):
  print("NEW ITERATION")
  print(i)
  print(j)
  df_temp = df_merge[(df_merge['store_nbr']==i[0])&(df_merge['family']==i[2])].reset_index(drop = True)
  df_temp["sales"] = np.log(df_temp["sales"]) #logarithmic normalization
  df_temp = df_temp.fillna(0) #filling null values
  df_temp = df_temp.replace([np.inf, -np.inf], 0) #filling values when log(0) takes place which cannot be computed
  df_test_temp =  df_test[(df_test['store_nbr']==i[0])&(df_test['family']==i[2])].reset_index(drop = True)
  if len(df_test_temp)>0:
    arma_model = ARMA(df_temp["sales"], order=(i[3],j[3])).fit() #fitting the model
    df_test_temp["sales"] = np.exp(arma_model.predict(start=len(df_temp), end=(len(df_temp)+len(df_test_temp)-1), dynamic=False).reset_index(drop=True).values)
    df_final_arma_v2 = pd.concat([df_final_arma_v2, df_test_temp])

In [ ]:
df_sample = pd.read_csv("/content/drive/MyDrive/ML Project /Time Series Analysis/Data/sample_submission.csv")
# df_sample["sales"] = df_final["pred"].values
df_sample_merge = pd.merge(df_sample["id"], df_final_arma_v2[['id','sales']], how='left', on='id')
df_sample_merge.head()